In [1]:
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import yfinance as yf

In [2]:
def get_price(ticker, start_date, end_date):
    """Return a DataFrame with price information (open, high, low, close, adjusted close, and volume) for the ticker between the specified dates."""
    df = yf.download(ticker, start_date, end_date, progress=False)
    df.reset_index(inplace=True)

    return df

In [3]:
def get_closed_dates(df):
    """Return a list containing all dates on which the stock market was closed."""
    # Create a dataframe that contains all dates from the start until today.
    timeline = pd.date_range(start=df['Date'].iloc[0], end=df['Date'].iloc[-1])

    # Create a list of the dates existing in the dataframe.
    df_dates = [day.strftime('%Y-%m-%d') for day in pd.to_datetime(df['Date'])]

    # Finally, determine which dates from the 'timeline' do not exist in our dataframe.
    closed_dates = [
        day for day in timeline.strftime('%Y-%m-%d').tolist()
        if not day in df_dates
    ]

    return closed_dates


## `get_MACD()`

In [4]:
def get_MACD(df, column='Adj Close'):
    """Return a new DataFrame with the MACD and related information (signal line and histogram)."""
    df['EMA-12'] = df[column].ewm(span=12, adjust=False).mean()
    df['EMA-26'] = df[column].ewm(span=26, adjust=False).mean()

    # MACD Indicator = 12-Period EMA − 26-Period EMA.
    df['MACD'] = df['EMA-12'] - df['EMA-26']

    # Signal line = 9-day EMA of the MACD line.
    df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # Histogram = MACD - Indicator.
    df['Histogram'] = df['MACD'] - df['Signal']

    return df


## `get_RSI()`

Source: https://tcoil.info/compute-rsi-for-stocks-with-python-relative-strength-index/

In [5]:
def get_RSI(df, column='Adj Close', time_window=14):
    """Return the RSI indicator for the specified time window."""
    diff = df[column].diff(1)

    # This preservers dimensions off diff values.
    up_chg = 0 * diff
    down_chg = 0 * diff

    # Up change is equal to the positive difference, otherwise equal to zero.
    up_chg[diff > 0] = diff[diff > 0]

    # Down change is equal to negative deifference, otherwise equal to zero.
    down_chg[diff < 0] = diff[diff < 0]

    # We set com = time_window-1 so we get decay alpha=1/time_window.
    up_chg_avg = up_chg.ewm(com=time_window - 1,
                            min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window - 1,
                                min_periods=time_window).mean()

    RS = abs(up_chg_avg / down_chg_avg)
    df['RSI'] = 100 - 100 / (1 + RS)

    return df

## `get_trading_strategy()`

In [6]:
def get_trading_strategy(df, column='Adj Close'):
    """Return the Buy/Sell signal on the specified (price) column (Default = 'Adj Close')."""
    buy_list, sell_list = [], []
    flag = False

    for i in range(0, len(df)):
        if df['MACD'].iloc[i] > df['Signal'].iloc[i] and flag == False:
            buy_list.append(df[column].iloc[i])
            sell_list.append(np.nan)
            flag = True

        elif df['MACD'].iloc[i] < df['Signal'].iloc[i] and flag == True:
            buy_list.append(np.nan)
            sell_list.append(df[column].iloc[i])
            flag = False

        else:
            buy_list.append(np.nan)
            sell_list.append(np.nan)

    df['Buy'] = buy_list
    df['Sell'] = sell_list

    return df

## `plot_candlestick_chart()`

In [7]:
def plot_candlestick_chart(fig, df, row, column=1, plot_EMAs=True, plot_strategy=True):
    """Return a graph object figure containing a Candlestick chart in the specified row."""
    fig.add_trace(go.Candlestick(x=df['Date'],
                                 open=df['Open'],
                                 high=df['High'],
                                 low=df['Low'],
                                 close=df['Close'],
                                 name='Candlestick Chart'),
                  row=row,
                  col=column)

    # If the boolean argument plot_EMAs is True, then show the line plots for the two exponential moving averages.
    if (plot_EMAs == True):
        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['EMA-12'],
                                 name='12-period EMA',
                                 line=dict(color='dodgerblue', width=2)),
                      row=row,
                      col=column)

        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['EMA-26'],
                                 name='26-period EMA',
                                 line=dict(color='whitesmoke', width=2)),
                      row=row,
                      col=column)

    if (plot_strategy == True):
        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['Buy'],
                                 name='Buy Signal',
                                 mode='markers',
                                 marker_symbol='triangle-up',
                                 marker=dict(size=9),
                                 line=dict(color='Lime')),
                      row=row,
                      col=column)

        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['Sell'],
                                 name='Sell Signal',
                                 mode='markers',
                                 marker_symbol='triangle-down',
                                 marker=dict(size=9, color='Yellow')),
                      row=row,
                      col=column)

    fig.update_xaxes(rangeslider={'visible': False})
    fig.update_yaxes(title_text='Price ($)', row=row, col=column)

    return fig


print('✔️ Function defined!')

✔️ Function defined!


## `plot_MACD()`

In [8]:
def plot_MACD(fig, df, row, column=1):
    """Return a graph object figure containing the MACD indicator, the signal line, and a histogram in the specified row."""
    df['Hist-Color'] = np.where(df['Histogram'] < 0, 'red', 'green')
    fig.add_trace(go.Bar(x=df['Date'],
                         y=df['Histogram'],
                         name='Histogram',
                         marker_color=df['Hist-Color'],
                         showlegend=True),
                  row=row,
                  col=column)

    fig.add_trace(go.Scatter(x=df['Date'],
                             y=df['MACD'],
                             name='MACD',
                             line=dict(color='darkorange', width=2)),
                  row=row,
                  col=column)

    fig.add_trace(go.Scatter(x=df['Date'],
                             y=df['Signal'],
                             name='Signal',
                             line=dict(color='cyan', width=2)),
                  row=row,
                  col=column)

    fig.update_yaxes(title_text='MACD', row=row, col=column)

    return fig


print('✔️ Function defined!')

✔️ Function defined!


## `plot_RSI()`

In [9]:
def plot_RSI(fig, df, row, column=1):
    """Return a graph object figure containing the RSI indicator in the specified row."""
    fig.add_trace(go.Scatter(x=df['Date'].iloc[30:],
                             y=df['RSI'].iloc[30:],
                             name='RSI',
                             line=dict(color='gold', width=2)),
                  row=row,
                  col=column)

    fig.update_yaxes(title_text='RSI', row=row, col=column)

    # Add one red horizontal line at 70% (overvalued) and green line at 30% (undervalued)
    for y_pos, color in zip([70, 30], ['Red', 'Green']):
        fig.add_shape(x0=df['Date'].iloc[1],
                      x1=df['Date'].iloc[-1],
                      y0=y_pos,
                      y1=y_pos,
                      type='line',
                      line=dict(color=color, width=2),
                      row=row,
                      col=column)

    # Add a text box for each line
    for y_pos, text, color in zip([64, 36], ['Overvalued', 'Undervalued'], ['Red', 'Green']):
        fig.add_annotation(x=df['Date'].iloc[int(df['Date'].shape[0] / 10)],
                           y=y_pos,
                           text=text,
                           font=dict(size=14, color=color),
                           bordercolor=color,
                           borderwidth=1,
                           borderpad=2,
                           bgcolor='lightsteelblue',
                           opacity=0.75,
                           showarrow=False,
                           row=row,
                           col=column)

    # Update the y-axis limits
    ymin = 25 if df['RSI'].iloc[30:].min() > 25 else df['RSI'].iloc[30:].min() - 5
    ymax = 75 if df['RSI'].iloc[30:].max() < 75 else df['RSI'].iloc[30:].max() + 5
    fig.update_yaxes(range=[ymin, ymax], row=row, col=column)

    return fig


print('✔️ Function defined!')

✔️ Function defined!


## `plot_volume()`

In [10]:
def plot_volume(fig, df, row, column=1):
    """Return a graph object figure containing the volume chart in the specified row."""
    fig.add_trace(go.Bar(x=df['Date'],
                         y=df['Volume'],
                         marker=dict(color='lightskyblue',
                                     line=dict(color='firebrick', width=0.1)),
                         showlegend=False,
                         name='Volume'),
                  row=row,
                  col=column)

    fig.update_xaxes(title_text='Date', row=row, col=column)
    fig.update_yaxes(title_text='Volume ($)', row=row, col=column)

    return fig


print('✔️ Function defined!')

✔️ Function defined!


<br>

# Getting the Data

For this project, we will retrieve data for [TESLA](https://www.tesla.com/en_gb)'s stock price. The user can choose another stock (or asset in general) by specifying the `ticker` variable. The user can also select the date range by changing the `no_years` variable.

In [11]:
ticker = 'TSLA'
no_years = 1

end_date = datetime.datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.datetime.now() -
              datetime.timedelta(days=no_years * 365)).strftime('%Y-%m-%d')

print('Ticker: {}'.format(ticker))
print('Start Date: ', start_date)
print('  End Date: ', end_date)

df = get_price(ticker, start_date, end_date)
closed_dates_list = get_closed_dates(df)

print('\n(Raw) Dataset Loaded!')
print('Last five rows:')
df.tail()

Ticker: TSLA
Start Date:  2022-11-19
  End Date:  2023-11-19

(Raw) Dataset Loaded!
Last five rows:


,Date,Open,High,Low,Close,Adj Close,Volume
245,2023-11-13,215.600006,225.399994,211.610001,223.710007,223.710007,140447600
246,2023-11-14,235.029999,238.139999,230.720001,237.410004,237.410004,149771600
247,2023-11-15,239.289993,246.699997,236.449997,242.839996,242.839996,150354000
248,2023-11-16,239.490005,240.880005,230.960007,233.589996,233.589996,136816800
249,2023-11-17,232.000000,237.389999,226.539993,234.300003,234.300003,142532800


Once the raw dataset is retrieved, we will apply the `get_MACD()` and `get_RSI()` functions to calculate the two indicators.

In [12]:
df = get_MACD(df)
df = get_RSI(df)

df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,EMA-12,EMA-26,MACD,Signal,Histogram,RSI
245,2023-11-13,215.600006,225.399994,211.610001,223.710007,223.710007,140447600,217.837342,224.508220,-6.670878,-8.803293,2.132415,49.111855
246,2023-11-14,235.029999,238.139999,230.720001,237.410004,237.410004,149771600,220.848521,225.463907,-4.615387,-7.965711,3.350325,57.217315
247,2023-11-15,239.289993,246.699997,236.449997,242.839996,242.839996,150354000,224.231825,226.751025,-2.519200,-6.876409,4.357209,59.940811
248,2023-11-16,239.490005,240.880005,230.960007,233.589996,233.589996,136816800,225.671543,227.257615,-1.586072,-5.818342,4.232270,53.672666
249,2023-11-17,232.000000,237.389999,226.539993,234.300003,234.300003,142532800,226.998999,227.779274,-0.780275,-4.810728,4.030453,54.069694


Finally, we will run the `get_trading_strategy()` function to get the buy and sell signals.

In [13]:
df = get_trading_strategy(df)

print('✔️ Final DataFrame is ready!')

✔️ Final DataFrame is ready!


<br>

# Creating the Dashboard

In [14]:
########## Plot the four plots ##########
fig = make_subplots(rows=4,
                    cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.005,
                    row_width=[0.2, 0.3, 0.3, 0.8])

fig = plot_candlestick_chart(fig,
                             df,
                             row=1,
                             plot_EMAs=True,
                             plot_strategy=True)
fig = plot_MACD(fig, df, row=2)
fig = plot_RSI(fig, df, row=3)
fig = plot_volume(fig, df, row=4)

########## Customise the figure ##########
# Update xaxis properties
fig.update_xaxes(rangebreaks=[dict(values=closed_dates_list)],
                 range=[df['Date'].iloc[0] - datetime.timedelta(days=3), df['Date'].iloc[-1] + datetime.timedelta(days=3)])

# Update basic layout properties (width&height, background color, title, etc.)
fig.update_layout(width=800,
                  height=800,
                  plot_bgcolor='#0E1117',
                  paper_bgcolor='#0E1117',
                  title={
                      'text': '{} - Stock Dashboard'.format(ticker),
                      'y': 0.98
                  },
                  hovermode='x unified',
                  legend=dict(orientation='h',
                              xanchor='left',
                              x=0.05,
                              yanchor='bottom',
                              y=1.003))

# Customize axis parameters
axis_lw, axis_color = 2, 'white'
fig.update_layout(xaxis1=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis1=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.update_layout(xaxis2=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis2=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.update_layout(xaxis3=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis3=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.update_layout(xaxis4=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis4=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.show()

<br>

# References

[1] [Technical Indicator](https://www.investopedia.com/terms/t/technicalindicator.asp#:~:text=Technical%20indicators%20are%20heuristic%20or,to%20predict%20future%20price%20movements.) by James Chen on [Investopedia](https://www.investopedia.com/) (Retrieved on Jun 11, 2021).

[2] [Technical Analyst](https://www.investopedia.com/terms/t/technical-analyst.asp) by Adam Hayes
on [Investopedia](https://www.investopedia.com/) (Retrieved on Jun 11, 2021).

[3] [Moving Average Convergence Divergence (MACD)](https://www.investopedia.com/terms/m/macd.asp) by Jason Fernando on [Investopedia](https://www.investopedia.com/) (Retrieved on Jun 11, 2021).

[4] [Relative Strength Index (RSI)](https://www.investopedia.com/terms/r/rsi.asp) by Jason Fernando on [Investopedia](https://www.investopedia.com/) (Retrieved on Jun 11, 2021). 

[5] [Compute RSI for stocks with python (Relative Strength Index)](https://tcoil.info/compute-rsi-for-stocks-with-python-relative-strength-index/) by Michal Vasulka on [tcoil.info](https://tcoil.info/) (Retrieved on Jun 10, 2021).

<br>

# Conclusions

 Our notebook came to an end! In summary, we learned how to use Python to import financial data, calculate two technical indicators, and visualise the information. 

<br>

Please consider <font size=+0 color="red"><b>upvoting</b></font> if you liked this notebook! Thank you! 😉